In [81]:
import pandas as pd
import csv
from bs4 import BeautifulSoup
import requests
from os import walk
import time
import random
import zipfile
import json
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
gene_list = []

gene_list = pd.read_csv("./dataset/to_scrape.txt", sep=', ')

gene_list = list(gene_list.columns)

fantom_db = pd.read_csv("./dataset/anno-fantom.csv")
snoDB = pd.read_csv("./dataset/snoDB_All_V2.0.csv",keep_default_na=False)

C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_2904\3895879587.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  gene_list = pd.read_csv("./dataset/to_scrape.txt", sep=', ')


In [12]:
def get_gene_links(gene_list):
    url = 'https://gene.disi.unitn.it/test/gene_history-z.php'
    data = {
        'org': 'Hs+(Homo+sapiens)',
        'tsize': '',
        'iter': '',
        'alpha': '0.05',
        'lgn': '',
        'submit': 'SEARCH'}
    missing_list = []

    for gene in gene_list:
        print(f"Looking for {gene}")
        data['lgn'] = gene
        soup = BeautifulSoup(requests.post(url, data = data).text, "lxml")

        if soup.find("div",{"id":"dialog-end"}) != None:
            missing_list.append(gene)
        else:
            table = soup.find("tbody")
            rows = table.find_all("tr")
            for row in rows:
                columns = row.find_all("td")
                if columns[2].string.split("-")[1] == gene:
                    downloaded_file = requests.get(f"https://gene.disi.unitn.it/test/gene_result-z.php?id={columns[0].string}_{columns[1].string}", allow_redirects=True)
                    open(f'./gene_expansions/{columns[0].string}_{columns[1].string}.zip', 'wb').write(downloaded_file.content) 
        time.sleep(random.uniform(1.5,2.0))

    return missing_list

missing_list = get_gene_links(gene_list)
print(f"Missing genes: {', '.join(missing_list)}")

Looking for SNORA1
Looking for SNORA12
Looking for SNORA14B
Looking for SNORA16A
Looking for SNORA21
Looking for SNORA23
Looking for SNORA24
Looking for SNORA32
Looking for SNORA38
Looking for SNORA44
Looking for SNORA48
Looking for SNORA49
Looking for SNORA52
Looking for SNORA53
Looking for SNORA57
Looking for SNORA61
Looking for SNORA63
Looking for SNORA64
Looking for SNORA65
Looking for SNORA70
Looking for SNORA71C
Looking for SNORA73A
Looking for SNORA73B
Looking for SNORA75
Looking for SNORA78
Looking for SNORA8
Looking for SNORD10
Looking for SNORD102
Looking for SNORD104
Looking for SNORD105
Looking for SNORD105B
Looking for SNORD108
Looking for SNORD110
Looking for SNORD111B
Looking for SNORD113-3
Looking for SNORD113-4
Looking for SNORD114-1
Looking for SNORD114-13
Looking for SNORD114-14
Looking for SNORD114-19
Looking for SNORD114-20
Looking for SNORD114-21
Looking for SNORD115-23
Looking for SNORD115-32
Looking for SNORD116-13
Looking for SNORD119
Looking for SNORD12
Lookin

In [72]:
filenames = next(walk('./gene_expansions/'), (None, None, []))[2]
final_df = pd.DataFrame()
cont = 0
for filename in filenames:
    archive = zipfile.ZipFile(f'./gene_expansions/{filename}', 'r')
    interactions = pd.read_csv(archive.open(f"{filename.split('.zip')[0]}.interactions"), skiprows=1).drop(['rank'], axis=1)
    if cont == 0:
        final_df = interactions
        cont += 1
    else:
        final_df = pd.concat(
        [final_df, interactions],
        axis=0,
        join="outer",
        ignore_index=True,
        keys=None,
        levels=None,
        names=None,
        verify_integrity=False,
        copy=True,
    )

In [73]:
final_df

,x,y,Fabs,Frel,intra
0,t053989,t168406,2012,1.000000,False
1,t053989,t074593,2006,1.000000,False
2,t053989,t174805,2012,1.000000,False
3,t014088,t053989,2006,1.000000,False
4,t053989,t186091,2012,1.000000,False
...,...,...,...,...,...
956026,t032906,t063635,3,0.001488,False
956027,t015160,t063635,3,0.001488,False
956028,t014018,t063635,3,0.001488,False
956029,t057602,t063635,3,0.001487,False


In [33]:
final_df.to_csv('merged_interactions.csv')

In [74]:
final_df = final_df[final_df['Frel'] > 0.05]
final_df.to_csv('merged_interactions_over_005.csv')

In [75]:
all_x = final_df.x.unique()
all_y = final_df.y.unique()

In [80]:
big_matrix = pd.DataFrame(index=all_y)
zeros = np.zeros(len(all_y))

for x in all_x:
    big_matrix[x] = zeros

TypeError: DataFrame.assign() takes 1 positional argument but 2 were given